In [15]:
import sys
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+

from pyspark.sql import SparkSession, functions, types
from pyspark.sql.functions import isnull, when, count, col
spark = SparkSession.builder.appName('home credit model training').getOrCreate()
assert spark.version >= '2.3'  # make sure we have Spark 2.3+
spark.sparkContext.setLogLevel('WARN')

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.stat import Statistics

import pandas as pd
import numpy as np

from ETL import get_dummies_spark

In [16]:
ready_train = spark.read.option("inferSchema", True).csv('/Users/michaelyang/Downloads/all_data/application_train.csv',
                                   header=True).limit(5000)

In [17]:
df = ready_train.drop('sk_id_curr').drop('target')

In [21]:
features = df.rdd.map(lambda row: row[0:])
a = df.rdd
a.collect()

[Row(NAME_CONTRACT_TYPE='Cash loans', CODE_GENDER='M', FLAG_OWN_CAR='N', FLAG_OWN_REALTY='Y', CNT_CHILDREN=0, AMT_INCOME_TOTAL=202500.0, AMT_CREDIT=406597.5, AMT_ANNUITY=24700.5, AMT_GOODS_PRICE=351000.0, NAME_TYPE_SUITE='Unaccompanied', NAME_INCOME_TYPE='Working', NAME_EDUCATION_TYPE='Secondary / secondary special', NAME_FAMILY_STATUS='Single / not married', NAME_HOUSING_TYPE='House / apartment', REGION_POPULATION_RELATIVE=0.018801, DAYS_BIRTH=-9461, DAYS_EMPLOYED=-637, DAYS_REGISTRATION=-3648.0, DAYS_ID_PUBLISH=-2120, OWN_CAR_AGE=None, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=1, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=1.0, REGION_RATING_CLIENT=2, REGION_RATING_CLIENT_W_CITY=2, WEEKDAY_APPR_PROCESS_START='WEDNESDAY', HOUR_APPR_PROCESS_START=10, REG_REGION_NOT_LIVE_REGION=0, REG_REGION_NOT_WORK_REGION=0, LIVE_REGION_NOT_WORK_REGION=0, REG_CITY_NOT_LIVE_CITY=0, REG_CITY_NOT_WORK_CITY=0, LIVE_CITY_NOT_WORK_CITY=0, ORGANIZATION_TY

In [49]:
ready_train = ready_train.drop('SK_ID_CURR')

In [50]:
len(ready_train.columns)

249

In [3]:
ready_train = ready_train.withColumn('CREDIT_INCOME_PERCENT', ready_train.AMT_CREDIT / ready_train.AMT_INCOME_TOTAL)
ready_train = ready_train.withColumn('ANNUITY_INCOME_PERCENT', ready_train.AMT_ANNUITY/ ready_train.AMT_INCOME_TOTAL)
ready_train = ready_train.withColumn('CREDIT_TERM', ready_train.AMT_ANNUITY/ ready_train.AMT_CREDIT)

In [3]:
train_cat_encoded = get_dummies_spark(ready_train,'SK_ID_CURR','ready_train')
numerical_feats = [f for f, t in ready_train.dtypes if t != 'string']
train_num_df = ready_train.select(numerical_feats)
ready_train = train_cat_encoded.join(train_num_df,on = 'SK_ID_CURR')

In [4]:
df = ready_train.drop('SK_ID_CURR').drop('TARGET')
col_names = df.columns
features = df.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
#corr_df = pd.DataFrame(corr_mat)
#corr_df.index, corr_df.columns = col_names, col_names
corr_mat

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1188, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1014, in send_command
    response = connection.send_command(command)
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1193, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:59985)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-887c531baff3>", line 4, in <module>
    corr_mat=Statistics.corr(features, method="pearson")
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/stat/_statistics.py", line 155, in corr
    return callMLlibFunc("corr", x.map(_convert_to_vector), method).toArray()
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/common.py", line 130, in callMLlibFunc
    return callJavaFunc(sc, api, *args)
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/common.py", line 123, in callJavaFunc
    return _java2py(sc, func(*args))
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1286, in __call__
    answer, s

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:59985)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-887c531baff3>", line 4, in <module>
    corr_mat=Statistics.corr(features, method="pearson")
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/stat/_statistics.py", line 155, in corr
    return callMLlibFunc("corr", x.map(_convert_to_vector), method).toArray()
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/common.py", line 130, in callMLlibFunc
    return callJavaFunc(sc, api, *args)
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/common.py", line 123, in callJavaFunc
    return _java2py(sc, func(*args))
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1286, in __call__
    answer, s

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:59985)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-887c531baff3>", line 4, in <module>
    corr_mat=Statistics.corr(features, method="pearson")
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/stat/_statistics.py", line 155, in corr
    return callMLlibFunc("corr", x.map(_convert_to_vector), method).toArray()
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/common.py", line 130, in callMLlibFunc
    return callJavaFunc(sc, api, *args)
  File "/usr/local/lib/python3.7/site-packages/pyspark/mllib/common.py", line 123, in callJavaFunc
    return _java2py(sc, func(*args))
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1286, in __call__
    answer, s

Py4JError: An error occurred while calling o687.corr

In [9]:
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
to_drop

In [12]:
for col in ready_train.columns:
    if col in to_drop:
        ready_train.drop(col)
print(ready_train.columns)

In [7]:

temp = ready_train.select([count(when(isnull(c), c)).alias(c) for c in ready_train.columns]).collect()
size = ready_train.count()
threshold = 0.75 * size
to_drop = []
for col in ready_train.columns:
    if temp[0][col] > threshold:
        to_drop.append(col)
to_drop

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54096)
Traceback (most recent call last):
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michaelyang/Library/Python/3.7/lib/python/site-packages/py4j/java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54096)

In [61]:
for col in ready_train.columns:
    if col in to_drop:
        ready_train.drop(col)
print(ready_train.columns)

[]

In [5]:
# assembler
features = list(set(ready_train.columns) - set(['SK_ID_CURR','TARGET']))
feature_assembler = VectorAssembler(inputCols=features ,outputCol='features')
# classifier
classifier = RandomForestClassifier(labelCol='TARGET',maxBins=60,maxDepth=6)
# pipeline
credit_pipeline = Pipeline(stages=[feature_assembler, classifier])
# fit and see feature importance
credit_model = credit_pipeline.fit(ready_train.na.fill(-999))

In [6]:
feature_importances = pd.DataFrame({'feature': features, 
                                    'importance': credit_model.stages[-1].featureImportances})\
    .sort_values('importance', ascending = False)
zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
zero_features

['e_ready_train_ORGANIZATION_TYPE_Medicine',
 'e_ready_train_ORGANIZATION_TYPE_Industry: type 8',
 'e_ready_train_ORGANIZATION_TYPE_Industry: type 6',
 'e_ready_train_NAME_FAMILY_STATUS_Widow',
 'e_ready_train_ORGANIZATION_TYPE_Realtor',
 'FLAG_DOCUMENT_7',
 'e_ready_train_ORGANIZATION_TYPE_Military',
 'e_ready_train_NAME_INCOME_TYPE_Unemployed',
 'e_ready_train_ORGANIZATION_TYPE_Transport: type 4',
 'e_ready_train_EMERGENCYSTATE_MODE_Yes',
 'e_ready_train_ORGANIZATION_TYPE_Trade: type 5',
 'e_ready_train_FONDKAPREMONT_MODE_reg oper spec account',
 'e_ready_train_OCCUPATION_TYPE_Waiters/barmen staff',
 'e_ready_train_NAME_EDUCATION_TYPE_Academic degree',
 'e_ready_train_ORGANIZATION_TYPE_Bank',
 'e_ready_train_NAME_HOUSING_TYPE_Co-op apartment',
 'e_ready_train_ORGANIZATION_TYPE_Mobile',
 'e_ready_train_NAME_TYPE_SUITE_unknown',
 'e_ready_train_ORGANIZATION_TYPE_Construction',
 'e_ready_train_ORGANIZATION_TYPE_Other',
 'e_ready_train_NAME_INCOME_TYPE_Pensioner',
 'e_ready_train_FONDKAP

In [7]:
for col in ready_train.columns:
    if col in zero_features:
        ready_train.drop(col)

In [9]:
len(ready_train.columns)

250

In [10]:
features = list(set(ready_train.columns) - set(['SK_ID_CURR', 'TARGET']))
feature_assembler = VectorAssembler(inputCols=features, outputCol='features')
classifier = GBTClassifier(labelCol='TARGET', maxBins=60)
pipeline = Pipeline(stages=[feature_assembler, classifier])
grid = ParamGridBuilder().addGrid(classifier.maxDepth,[4,5]).build()
evaluator = BinaryClassificationEvaluator(labelCol='TARGET')
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=evaluator, numFolds= 5)
cv_model = cv.fit(ready_train.fillna(-999))

In [28]:
model = cv_model.bestModel

In [33]:
cv_model.avgMetrics

[0.7424605962889839, 0.7264007557875325]

In [41]:
print(model.stages[1].extractParamMap())

{Param(parent='GBTClassifier_46a099b5b9ac74a2a2bb', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False, Param(parent='GBTClassifier_46a099b5b9ac74a2a2bb', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10, Param(parent='GBTClassifier_46a099b5b9ac74a2a2bb', name='featuresCol', doc='features column name'): 'features', Param(parent='GBTClassifier_46a099b5b9ac74a2a2bb', name='labelCol', doc='label column name'): 'TARGET', Param(parent='GBTClassifier_46a099b5b9ac74a2a2bb', name='lossType', doc='Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic'): 'logistic', Param(pare

In [6]:
previous_application = spark.read.option("inferSchema", True)\
    .csv('/Users/michaelyang/Downloads/all_data/previous_application.csv',header=True).limit(1000)

In [13]:
df = previous_application
pivot_cols = [f for f, t in df.dtypes if t == 'string']
keys = pivot_cols + ['SK_ID_PREV']
before = df.select(keys).na.fill('unknown')
pivotDF = before.groupBy(keys).pivot(pivot_cols[3]).count()

In [14]:
pivotDF.columns

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION',
 'SK_ID_PREV',
 'Building a house or an annex',
 'Car repairs',
 'Everyday expenses',
 'Journey',
 'Medicine',
 'Other',
 'Payments on other loans',
 'Purchase of electronic equipment',
 'Repairs',
 'Urgent needs',
 'XAP',
 'XNA']